# Personal Finance

Date: 5/1/2024

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Budget

### Income

In [2]:
base_salary = 185_000

In [3]:
bonus_factor = 0.12 # standard base
annual_bonus = base_salary * bonus_factor
annual_bonus

22200.0

In [4]:
share_price = 300.00
num_shares = 202
annual_equity = share_price * num_shares
annual_equity

60600.0

In [5]:
tax_rate = 0.34
bonus_tax_rate = 0.42

In [6]:
other_monthly_income = 319 # ends Aug 2024

In [7]:
annual_compensation = base_salary + annual_equity + annual_bonus
annual_compensation

267800.0

In [8]:
total_annual_income = annual_compensation + 8 * other_monthly_income # for 2024
total_annual_income

270352.0

In [9]:
net_annual_income = base_salary * (1 - tax_rate) + (annual_equity + annual_bonus) * (1 - bonus_tax_rate) + 8 * other_monthly_income
net_annual_income

172676.0

In [10]:
monthly_salary = base_salary / 12
round(monthly_salary, 2)

15416.67

In [11]:
total_monthly_income = base_salary / 12 + other_monthly_income
round(total_monthly_income, 2)

15735.67

In [12]:
net_monthly_income = base_salary * (1 - tax_rate) / 12 + other_monthly_income
round(net_monthly_income, 2)

10494.0

In [13]:
total_per_paycheck = base_salary / 12 / 2
round(total_per_paycheck, 2)

7708.33

In [14]:
net_per_paycheck = total_per_paycheck * (1 - tax_rate)
round(net_per_paycheck, 2)

5087.5

In [15]:
effective_total_monthly_income = total_monthly_income + annual_bonus / 12 + annual_equity / 12
round(effective_total_monthly_income, 2)

22635.67

In [16]:
effective_net_monthly_income = net_annual_income / 12
round(effective_net_monthly_income, 2)

14389.67

### Costs

In [17]:
fixed_costs = sum([
    3000, # rent
    170, # utilities
    50, # health insurance
    120, # car insurance
    15 * 2 + 12 * 2, # subscriptions
    500, # student loans
    1249, # car loan
])

fixed_costs

5143

In [18]:
variable_costs_sc = sum([
    30 * 30, # food + coffee + alcohol
    100, # gas
    450 + 150, # travel (GSP-SFO airfare + 1 hotel)
    200, # misc
])

variable_costs_sc

1800

In [19]:
variable_costs_ca = sum([
    50 * 30, # food + coffee + alcohol
    100, # gas
    450 + 4 * 100, # travel (GSP-SFO airfare + 4 hotels)
    200, # misc
])

variable_costs_ca

2650

In [20]:
total_monthly_costs_sc = fixed_costs + variable_costs_sc
total_monthly_costs_sc

6943

In [21]:
total_monthly_costs_ca = fixed_costs + variable_costs_ca
total_monthly_costs_ca

7793

In [22]:
avg_total_monthly_costs = (total_monthly_costs_sc + total_monthly_costs_ca) / 2
avg_total_monthly_costs

7368.0

In [23]:
total_annual_costs = total_monthly_costs_sc * 6 + total_monthly_costs_ca * 6
total_annual_costs

88416

### Profits

In [24]:
avg_monthly_profit = net_monthly_income - avg_total_monthly_costs
round(avg_monthly_profit, 2)

3126.0

In [25]:
avg_effective_monthly_profit = effective_net_monthly_income - avg_total_monthly_costs
round(avg_effective_monthly_profit, 2)

7021.67

In [26]:
annual_profit = net_annual_income - total_annual_costs
annual_profit

84260.0

## Student Loan

In [27]:
df_student = pd.DataFrame(data=[
    ['AA', 9_917.64, 3.150],
    ['AB', 0.00, 0.0],
    ['AC', 6_029.35, 3.610],
    ['AD', 9_171.32, 3.610],
    ['AE', 0.00, 0.0],
    ['AF', 5_388.90, 4.410],
    ['AG', 0.00, 5.590],
    ['AH', 9_091.17, 5.060],
    ['AI', 0.00, 0.0],
    ['AJ', 0.00, 0.0],
], columns=['group', 'principle', 'apr'])
df_student['monthly_interest'] = (df_student['principle'] * df_student['apr'] / 12 / 100).round(2)
df_student

,group,principle,apr,monthly_interest
0,AA,9917.64,3.15,26.03
1,AB,0.00,0.00,0.00
2,AC,6029.35,3.61,18.14
3,AD,9171.32,3.61,27.59
4,AE,0.00,0.00,0.00
5,AF,5388.90,4.41,19.80
6,AG,0.00,5.59,0.00
7,AH,9091.17,5.06,38.33
8,AI,0.00,0.00,0.00
9,AJ,0.00,0.00,0.00


In [28]:
effective_student_apr = np.dot(df_student['principle'] / df_student['principle'].sum(), df_student['apr'])
round(effective_student_apr, 3)

3.937

In [29]:
total_student_principle = (df_student['principle'] + df_student['monthly_interest']).sum()
round(total_student_principle, 2)

39728.27

In [30]:
def amortization_schedule(df, min_payment, extra_payments=[], start_at=0):
    df = df[df['principle'] > 0].copy()
    month = start_at
    amortization_records = [{
            'month': month,
            'total_payment': 0,
            'interest_paid': 0,
            'principal_paid': 0,
            'remaining_balance': df['principle'].sum()
    }]
    total_principal = df['principle'].sum()
    while total_principal > 0:
        month += 1
        monthly_interest = df['monthly_interest'].sum()
        total_payment = min_payment
        for extra_payment in extra_payments:
            if month >= extra_payment['shift_by'] and (month - extra_payment.get('shift_by', 0)) % extra_payment['num_months'] == 0:
                total_payment += extra_payment['amount']
        principal_payment = max(0, total_payment - monthly_interest)
        if total_principal + monthly_interest < total_payment:
            principal_payment = total_principal
            min_payment = principal_payment + monthly_interest
        total_principal -= principal_payment
        amortization_records.append({
            'month': month,
            'total_payment': total_payment,
            'interest_paid': monthly_interest,
            'principal_paid': principal_payment,
            'remaining_balance': total_principal
        })
        df['payment_proportion'] = df['principle'] / df['principle'].sum()
        df['principle'] -= df['payment_proportion'] * principal_payment
        df['monthly_interest'] = (df['principle'] * df['apr'] / 12 / 100).round(2)
        if total_principal <= 1:
            break
    return pd.DataFrame(amortization_records)

In [31]:
min_student_loan_payment = 500
extra_payments = [
    {'num_months': 1, 'amount': 500, 'shift_by': 0},
    {'num_months': 3, 'amount': 9000, 'shift_by': 1},
    {'num_months': 6, 'amount': 6000, 'shift_by': 7},
]
amortization_df = amortization_schedule(df_student, min_student_loan_payment, extra_payments)
amortization_df

,month,total_payment,interest_paid,principal_paid,remaining_balance
0,0,0,0.00,0.00,39598.38
1,1,10000,129.89,9870.11,29728.27
2,2,1000,97.52,902.48,28825.79
3,3,1000,94.56,905.44,27920.35
4,4,10000,91.59,9908.41,18011.94
5,5,1000,59.09,940.91,17071.03
6,6,1000,56.00,944.00,16127.03
7,7,16000,52.91,15947.09,179.94
8,8,1000,0.59,179.94,0.00


In [32]:
num_months_remaining_student = amortization_df['month'].iloc[-1]
num_months_remaining_student

8

In [33]:
num_months_remaining_student // 12, num_months_remaining_student % 12

(0, 8)

In [34]:
leftover_student_payment = amortization_df['total_payment'].iloc[-1] - amortization_df['remaining_balance'].iloc[-2]
round(leftover_student_payment, 2)

820.06

## Car Loan

In [35]:
car_payment = 1249.36
car_principle = 39_032.92
car_apr = 1.99

remaining_car_principle = car_principle - num_months_remaining_student * car_payment - leftover_student_payment
df_car = pd.DataFrame(data=[[remaining_car_principle, car_apr]], columns=['principle', 'apr'])
df_car['monthly_interest'] = (df_car['principle'] * df_car['apr'] / 12 / 100).round(2)
df_car

,principle,apr,monthly_interest
0,28217.98,1.99,46.79


In [36]:
extra_payments = [
    {'num_months': 1, 'amount': 2000, 'shift_by': 0},
    {'num_months': 3, 'amount': 9000, 'shift_by': 1},
    {'num_months': 6, 'amount': 6000, 'shift_by': 1},
]
amortization_df = amortization_schedule(df_car, car_payment, extra_payments, start_at=num_months_remaining_student)
amortization_df

,month,total_payment,interest_paid,principal_paid,remaining_balance
0,8,0.00,0.00,0.00,28217.98
1,9,3249.36,46.79,3202.57,25015.41
2,10,12249.36,41.48,12207.88,12807.53
3,11,3249.36,21.24,3228.12,9579.41
4,12,3249.36,15.89,3233.47,6345.94
5,13,18249.36,10.52,6345.94,0.00


In [37]:
num_months_remaining_car = amortization_df['month'].iloc[-1]
num_months_remaining_car

13

In [38]:
num_months_remaining_car // 12, num_months_remaining_car % 12

(1, 1)

In [39]:
leftover_car_payment = amortization_df['total_payment'].iloc[-1] - amortization_df['remaining_balance'].iloc[-2]
round(leftover_car_payment, 2)

11903.42

## Timeline

In [40]:
num_months_loans = num_months_remaining_student + num_months_remaining_car - num_months_remaining_student
num_months_loans

13

In [41]:
savings_target = 50_000
current_balance = 15_000 + leftover_car_payment
monthly_profit_no_loans = avg_effective_monthly_profit + min_student_loan_payment + car_payment - other_monthly_income
round(monthly_profit_no_loans, 2)

8452.03

In [42]:
num_months_to_save = round((savings_target - current_balance) / monthly_profit_no_loans)
num_months_to_save

3

In [43]:
num_months_remaining = num_months_loans + num_months_to_save
num_months_remaining

16

In [44]:
# Student Loans Paid: Dec-Jan 2024
# Car Paid: Apr-May 2025
# Savings Met: July-Sep 2025
# depends mainly on how much extra paid per month (which depends on spending)